In [1]:
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
conf = SparkConf().setAppName("NewApp").setMaster("local")
sc = SparkContext(conf = conf)
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:46:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 08:46:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/22 08:46:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
#create dataframes upon file import

cusdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/customer.csv")

In [6]:
cusdf.show()

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|        1|Customer#000000001|   IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|
|        2|Customer#000000002|XSTf4,NCwDVaWNe6t...|         13|23-768-687-3665|   121.65|  AUTOMOBILE|l accounts. blith...|
|        3|Customer#000000003|        MG9kdTD2WBHm|          1|11-719-748-3364|  7498.12|  AUTOMOBILE| deposits eat sly...|
|        4|Customer#000000004|         XxVSJsLAGtn|          4|14-128-190-5944|  2866.83|   MACHINERY| requests. final,...|
|        5|Customer#000000005|KvpyuHCplrB84WgAi...|          3|13-750-942-6364|   794.47|   HOUSEHOLD|n accounts will h...|
|       

In [7]:
linedf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/lineitem.csv")
nationdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/nation.csv")
orderdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/orders.csv")
partdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/part.csv")
partsuppdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/partsupp.csv")
regiondf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/region.csv")
supplierdf = spark.read.format("csv").option("delimiter","\t").option("header",True).load("/home/labuser/Downloads/PartSupply/supplier.csv")

In [14]:
supplierdf.show()

+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|S_SUPPKEY|            S_NAME|           S_ADDRESS|S_NATIONKEY|        S_PHONE|S_ACCTBAL|           S_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|        1|Supplier#000000001| N kD4on9OM Ipw3,...|         17|27-918-335-1736|  5755.94|each slyly above ...|
|        2|Supplier#000000002|89eJ5ksX3ImxJQBvx...|          5|15-679-861-2259|  4032.68| slyly bold instr...|
|        3|Supplier#000000003|q1,G3Pj6OjIuUYfUo...|          1|11-383-516-1199|  4192.40|blithely silent r...|
|        4|Supplier#000000004|Bk7ah4CK8SYQTepEm...|         15|25-843-787-7479|  4641.08|riously even requ...|
|        5|Supplier#000000005|   Gcdm2rJRzl5qlTVzc|         11|21-151-690-3663|  -283.84|. slyly regular p...|
|        6|Supplier#000000006|        tQxuVm7s7CnK|         14|24-696-997-4969|  1365.79|final accounts. r...|
|

In [16]:
cusdf.printSchema()

root
 |-- C_CUSTKEY: string (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: string (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: string (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



In [22]:
newdf = cusdf.join(orderdf, cusdf.C_CUSTKEY == orderdf.O_CUSTKEY, how = "inner").select('C_CUSTKEY', 'C_Name', orderdf.O_ORDERKEY)
newdf.show()

+---------+------------------+----------+
|C_CUSTKEY|            C_Name|O_ORDERKEY|
+---------+------------------+----------+
|      370|Customer#000000370|         1|
|      781|Customer#000000781|         2|
|     1234|Customer#000001234|         3|
|     1369|Customer#000001369|         4|
|      445|Customer#000000445|         5|
|      557|Customer#000000557|         6|
|      392|Customer#000000392|         7|
|     1301|Customer#000001301|        32|
|      670|Customer#000000670|        33|
|      611|Customer#000000611|        34|
|     1276|Customer#000001276|        35|
|     1153|Customer#000001153|        36|
|      862|Customer#000000862|        37|
|     1249|Customer#000001249|        38|
|      818|Customer#000000818|        39|
|      322|Customer#000000322|        64|
|      163|Customer#000000163|        65|
|     1292|Customer#000001292|        66|
|      568|Customer#000000568|        67|
|      286|Customer#000000286|        68|
+---------+------------------+----

## Joins

In [31]:
cus_ord = cusdf.join(orderdf, cusdf.C_CUSTKEY == orderdf.O_CUSTKEY, how="inner")
#cus_ord.show()

## GroupBy and Aggregations

In [38]:
#cus_ord = cus_ord.selectExpr('CAST(O_TOTALPRICE as Integer)')
cus_ord.groupBy('C_CUSTKEY').sum('O_TOTALPRICE')

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `C_CUSTKEY` cannot be resolved. Did you mean one of the following? [`O_TOTALPRICE`].;
'Aggregate ['C_CUSTKEY], ['C_CUSTKEY, sum(O_TOTALPRICE#1313) AS sum(O_TOTALPRICE)#1317L]
+- Project [cast(O_TOTALPRICE#244 as int) AS O_TOTALPRICE#1313]
   +- Join Inner, (C_CUSTKEY#92 = O_CUSTKEY#242)
      :- Relation [C_CUSTKEY#92,C_NAME#93,C_ADDRESS#94,C_NATIONKEY#95,C_PHONE#96,C_ACCTBAL#97,C_MKTSEGMENT#98,C_COMMENT#99] csv
      +- Relation [O_ORDERKEY#241,O_CUSTKEY#242,O_ORDERSTATUS#243,O_TOTALPRICE#244,O_ORDERDATE#245,O_ORDERPRIORITY#246,O_CLERK#247,O_SHIPPRIORITY#248,O_COMMENT#249] csv


In [33]:
cus_ord.printSchema()

root
 |-- C_CUSTKEY: string (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: string (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: string (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)
 |-- O_ORDERKEY: string (nullable = true)
 |-- O_CUSTKEY: string (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: string (nullable = true)
 |-- O_ORDERDATE: string (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: string (nullable = true)
 |-- O_COMMENT: string (nullable = true)



## Parquet

In [34]:
orderdf.printSchema()

root
 |-- O_ORDERKEY: string (nullable = true)
 |-- O_CUSTKEY: string (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: string (nullable = true)
 |-- O_ORDERDATE: string (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: string (nullable = true)
 |-- O_COMMENT: string (nullable = true)



## Coalesce